[View in Colaboratory](https://colab.research.google.com/github/KushalVenkatesh/botkit/blob/master/Extractive_Text_Summarization_using_TextRank_Algorithm.ipynb)

In [0]:
!pwd

/d/python2


In [0]:
!python --version

Python 2.7.15 :: Anaconda, Inc.


The Extractive versus Abstractive

The extractive approach entails selecting the X most representative sentences that best cover the whole information expressed by the original text. This is the most popular approach, especially because it’s a much easier task than the abstractive approach.

In the abstractive approach, we basically build a summary of the text, in the way a human would build one. We pick ideas from several paragraphs or sentences, build readable sentences and present them in a concise form. 

The text summarization is mainly useful because it condenses information for easier consumption and analysis. 

The TextRank Algorithm
Here is a very popular and accurate extractive text summarization algorithm. It’s called TextRank. Before diving into TextRank algorithm, we must first make sure we understand the PageRank algorithm, because it’s the foundation of TextRank.

We might have already heard of PageRank since it’s used heavily on the service we use most on the web: Google Search. It’s used to compute the rank of web pages, but funny enough, it’s not named after its use (ranking pages) but rather after its creator: Larry Page, one of Google’s founders. Obviously, the algorithm used inside Google is almost transfigured to be recognised as PageRank. This is due to the fact that inside Google the algorithm works at an incredible scale. We’ll be focusing here only on the original, plain PageRank algorithm. The algorithm itself is very interesting and worth our attention. 

Here are the fundamentals of it:

The main insight: Important pages are linked by important pages (a recurrent definition)
The PageRank value of a page is essentially the probability of a user visiting that page

Let’s now define the variables we’ll be using:

We have N pages with links between them
P = PageRank vector (P[i] = the pagerank of page i)

A[i][j] is the probability of the user transitioning from page i to page j
A[i][j] is initialized with 1.0 / outbound_links_count(from_node=i) if there’s a link between i and j and 0 otherwise

If a page i has no outbound links, then A[i][j] = 1.0/N. Assign equal probability to transitioning to any page. Pages with no outbound links are called dangling pages.

Let’s write a basic implementation of PageRank. First, we’ll write a simple 10 web page graph to apply the algorithm on:

In [0]:
links = {
    'webpage-1': set(['webpage-2', 'webpage-4', 'webpage-5', 'webpage-6', 'webpage-8', 'webpage-9', 'webpage-10']),
    'webpage-2': set(['webpage-5', 'webpage-6']),
    'webpage-3': set(['webpage-10']),
    'webpage-4': set(['webpage-9']),
    'webpage-5': set(['webpage-2', 'webpage-4']),
    'webpage-6': set([]), # dangling page
    'webpage-7': set(['webpage-1', 'webpage-3', 'webpage-4']),
    'webpage-8': set(['webpage-1']),
    'webpage-9': set(['webpage-1', 'webpage-2', 'webpage-3', 'webpage-8', 'webpage-10']),
    'webpage-10': set(['webpage-2', 'webpage-3', 'webpage-8', 'webpage-9']),
}
 

Next, we will write a function that builds an index of the webpages and assigns them to a numeric index. We will use this to build an adjacency matrix.

In [0]:
def build_index(links):
    website_list = links.keys()
    return {website: index for index, website in enumerate(website_list)}
 
website_index = build_index(links)
print(website_index)
# {'webpage-10': 3, 'webpage-9': 0, 'webpage-8': 1, 'webpage-1': 2, 'webpage-3': 4, 'webpage-2': 5, 'webpage-5': 6, 'webpage-4': 7, 'webpage-7': 8, 'webpage-6': 9}
 


{'webpage-10': 3, 'webpage-9': 0, 'webpage-8': 1, 'webpage-1': 2, 'webpage-3': 4, 'webpage-2': 5, 'webpage-5': 6, 'webpage-4': 7, 'webpage-7': 8, 'webpage-6': 9}


We need to feed a transition probability matrix to PageRank. A[i][j] is the probability of transitioning from page i to page j. Here’s how to build that:

In [0]:
import numpy as np
 
def build_transition_matrix(links, index):
    total_links = 0
    A = np.zeros((len(index), len(index)))
    for webpage in links:
        # dangling page
        if not links[webpage]:
            # Assign equal probabilities to transition to all the other pages
            A[index[webpage]] = np.ones(len(index)) / len(index)
        else:
            for dest_webpage in links[webpage]:
                total_links += 1
                A[index[webpage]][index[dest_webpage]] = 1.0 / len(links[webpage])
 
    return A
 
A = build_transition_matrix(links, website_index)
print(A)

[[0.         0.2        0.2        0.2        0.2        0.2
  0.         0.         0.         0.        ]
 [0.         0.         1.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.14285714 0.14285714 0.         0.14285714 0.         0.14285714
  0.14285714 0.14285714 0.         0.14285714]
 [0.25       0.25       0.         0.         0.25       0.25
  0.         0.         0.         0.        ]
 [0.         0.         0.         1.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.5        0.         0.         0.5       ]
 [0.         0.         0.         0.         0.         0.5
  0.         0.5        0.         0.        ]
 [1.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.33333333 0.         0.33333333 0.
  0.         0.33333333 0.         0.        ]
 [0.1        0.1        0

Now we’ll write the actual PageRank algorithm. The algorithm takes 2 parameters:
1. eps: stop the algorithm when the difference between 2 consecutive iterations is smaller or equal to eps
2. d: damping factor: With a probability of 1-d the user will simply pick a web page at random as the next destination, ignoring the link structure completely.

In [0]:
def pagerank(A, eps=0.0001, d=0.85):
    P = np.ones(len(A)) / len(A)
    while True:
        new_P = np.ones(len(A)) * (1 - d) / len(A) + d * A.T.dot(P)
        delta = abs(new_P - P).sum()
        if delta <= eps:
            return new_P
        P = new_P
 
results = pagerank(A)
 
print("Results:", results) # [ 0.13933698,  0.09044235,  0.1300934 ,  0.13148714,  0.08116465, 0.1305122 ,  0.09427366,  0.085402  ,  0.02301397,  0.09427366]
print(sum(results)) # 1.0
print([item[0] for item in sorted(enumerate(results), key=lambda item: -item[1])]) # [0, 3, 5, 2, 6, 9, 1, 7, 4, 8]

('Results:', array([0.13933698, 0.09044235, 0.1300934 , 0.13148714, 0.08116465,
       0.1305122 , 0.09427366, 0.085402  , 0.02301397, 0.09427366]))
1.0000000000000002
[0, 3, 5, 2, 6, 9, 1, 7, 4, 8]


In [0]:
!pip install nltk downloader

  Running setup.py bdist_wheel for downloader: started
  Running setup.py bdist_wheel for downloader: finished with status 'done'
  Stored in directory: C:\Users\kusha\AppData\Local\pip\Cache\wheels\b2\eb\46\10c34ef0b1d8fad0f28c2514e44a69502e58dc47debe15f419
Successfully built downloader


distributed 1.21.8 requires msgpack, which is not installed.
grin 1.2.1 requires argparse>=1.1, which is not installed.


From PageRank to TextRank:

We might be asking ourself: “How does PageRank help me with text summarization?”

Here’s the trick:

We consider sentences the equivalent of web pages
The probability of going from sentence A to sentence B is equal to the similarity of the 2 sentences
Apply the PageRank algorithm over this sentence graph

A few observations:

In the case of TextRank, the graph is symmetrical
It’s up to us to come up with a similarity measure and the algorithm’s performance depends strongly upon this measure
The algorithm is language agnostic. (It can become language dependent due to a language dependent implementation of the similarity measure)
It doesn’t require any training, it’s a totally unsupervised method (which we like)
Here are some conditions a good similarity measure has to obey:

0 <= similarity(A, B) <= 1
similarity(A, A) = 1
similarity(A, B) =/= 1 if A =/= B

A widely used measure in Natural Language Processing is the Cosine Similarity. The Cosine Similarity computes the cosine of the angle between 2 vectors. If the vectors are identical, the cosine is 1.0. If the vectors are orthogonal, the cosine is 0.0. This means the cosine similarity is a measure we can use.

NLTK implements cosine_distance, which is 1 - cosine_similarity. The concept of distance is opposite to similarity. Two identical vectors are located at 0 distance and are 100% similar.

Let’s write a good sentence similarity measure:

In [0]:
import nltk
nltk.download('stopwords')
from nltk.corpus import brown, stopwords
from nltk.cluster.util import cosine_distance
 
def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []
 
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
 
    all_words = list(set(sent1 + sent2))
 
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
 
    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
 
    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
 
    return 1 - cosine_distance(vector1, vector2)
 
# One out of 5 words differ => 0.8 similarity
print(sentence_similarity("This is a good sentence".split(), "This is a bad sentence".split()))
 
# One out of 2 non-stop words differ => 0.5 similarity
print(sentence_similarity("This is a good sentence".split(), "This is a bad sentence".split(), stopwords.words('english')))
 
# 0 out of 2 non-stop words differ => 1 similarity (identical sentences)
print(sentence_similarity("This is a good sentence".split(), "This is a good sentence".split(), stopwords.words('english')))
 
# Completely different sentences=> 0.0
print(sentence_similarity("This is a good sentence".split(), "I want to go to the market".split(), stopwords.words('english')))
 


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kusha\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
0.7999999999999998
0.4999999999999999
0.9999999999999998
0.0


Here let’s build the PageRank transition matrix by building the sentence similarity matrix:

In [0]:
import numpy as np
import nltk
nltk.download('brown')
#nltk.download()
from nltk.corpus import brown, stopwords
from nltk.cluster.util import cosine_distance

 
# Get a text from the Brown Corpus
sentences = brown.sents('ca01')
 
print(sentences)
# [[u'The', u'Fulton', u'County', u'Grand', u'Jury', u'said', u'Friday', u'an', u'investigation', u'of', u"Atlanta's", u'recent', u'primary', u'election', u'produced', u'``', u'no', u'evidence', u"''", u'that', u'any', u'irregularities', u'took', u'place', u'.'], [u'The', u'jury', u'further', u'said', u'in', u'term-end', u'presentments', u'that', u'the', u'City', u'Executive', u'Committee', u',', u'which', u'had', u'over-all', u'charge', u'of', u'the', u'election', u',', u'``', u'deserves', u'the', u'praise', u'and', u'thanks', u'of', u'the', u'City', u'of', u'Atlanta', u"''", u'for', u'the', u'manner', u'in', u'which', u'the', u'election', u'was', u'conducted', u'.'], ...]
 
print(len(sentences))  #  98
 
# get the english list of stopwords
stop_words = stopwords.words('english')
 
 
def build_similarity_matrix(sentences, stopwords=None):
    # Create an empty similarity matrix
    S = np.zeros((len(sentences), len(sentences)))
 
 
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2:
                continue
 
            S[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)
 
    # normalize the matrix row-wise
    for idx in range(len(S)):
        S[idx] /= S[idx].sum()
 
    return S
 
S = build_similarity_matrix(sentences, stop_words)    
print(S)

[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\kusha\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\brown.zip.
[[u'The', u'Fulton', u'County', u'Grand', u'Jury', u'said', u'Friday', u'an', u'investigation', u'of', u"Atlanta's", u'recent', u'primary', u'election', u'produced', u'``', u'no', u'evidence', u"''", u'that', u'any', u'irregularities', u'took', u'place', u'.'], [u'The', u'jury', u'further', u'said', u'in', u'term-end', u'presentments', u'that', u'the', u'City', u'Executive', u'Committee', u',', u'which', u'had', u'over-all', u'charge', u'of', u'the', u'election', u',', u'``', u'deserves', u'the', u'praise', u'and', u'thanks', u'of', u'the', u'City', u'of', u'Atlanta', u"''", u'for', u'the', u'manner', u'in', u'which', u'the', u'election', u'was', u'conducted', u'.'], ...]
98
[[0.         0.02171602 0.02438459 ... 0.01056602 0.02113203 0.0224139 ]
 [0.01642812 0.         0.00853223 ... 0.00646989 0.01940966 0.0137247 ]
 [0.0326456  0.01509956 0.

Now that we know how to compute the similarity matrix, we can compute the summary. For demonstration purposes, we’re using the first document from the Brown corpus. Here’s a rough version of the code:

In [0]:
from operator import itemgetter 
 
sentence_ranks = pagerank(S)
 
print(sentence_ranks)
 
# Get the sentences ordered by rank
ranked_sentence_indexes = [item[0] for item in sorted(enumerate(sentence_ranks), key=lambda item: -item[1])]
print(ranked_sentence_indexes)
 
# Suppose we want the 5 most import sentences
SUMMARY_SIZE = 5
SELECTED_SENTENCES = sorted(ranked_sentence_indexes[:SUMMARY_SIZE])
print(SELECTED_SENTENCES)
 
# Fetch the most important sentences
summary = itemgetter(*SELECTED_SENTENCES)(sentences)
 
# Print the actual summary
for sentence in summary:
    print(' '.join(sentence))

[0.01121618 0.01416586 0.00875955 0.01720666 0.01097157 0.00992082
 0.01200115 0.00168689 0.01309704 0.01454477 0.01097302 0.00816622
 0.00878952 0.00681878 0.01487246 0.00980958 0.01631003 0.01077275
 0.01016464 0.00164935 0.01201909 0.01345858 0.01054308 0.01246479
 0.00189483 0.00848166 0.01189495 0.00226414 0.00871032 0.01280831
 0.01201529 0.0091875  0.0132896  0.01413383 0.0083716  0.01075297
 0.01131163 0.00866148 0.00781889 0.00798044 0.01358306 0.00816582
 0.00787996 0.00986716 0.01055781 0.01245516 0.00559471 0.00901723
 0.01349356 0.01029195 0.00615882 0.00646616 0.00680286 0.01227652
 0.00915185 0.01146637 0.01031809 0.00676416 0.00947115 0.00730678
 0.00687184 0.0022958  0.01043746 0.00837449 0.00795457 0.00182517
 0.0082859  0.00822529 0.01303836 0.00667884 0.00853634 0.01013017
 0.00852789 0.01224895 0.00630303 0.01017482 0.01098564 0.01494719
 0.00789689 0.01391384 0.01242932 0.0017009  0.01639154 0.01258241
 0.01145775 0.01336677 0.01761915 0.01135941 0.01403542 0.0138

In [0]:
Let’s bundle all this code in a nice & neat function:

In [0]:
def textrank(sentences, top_n=5, stopwords=None):
    """
    sentences = a list of sentences [[w11, w12, ...], [w21, w22, ...], ...]
    top_n = how may sentences the summary should contain
    stopwords = a list of stopwords
    """
    S = build_similarity_matrix(sentences, stop_words) 
    sentence_ranks = pagerank(S)
 
    # Sort the sentence ranks
    ranked_sentence_indexes = [item[0] for item in sorted(enumerate(sentence_ranks), key=lambda item: -item[1])]
    selected_sentences = sorted(ranked_sentence_indexes[:top_n])
    summary = itemgetter(*SELECTED_SENTENCES)(sentences)
    return summary
 
for idx, sentence in enumerate(textrank(sentences, stopwords=stopwords.words('english'))):
    print("%s. %s" % ((idx + 1), ' '.join(sentence)))
 
# 1. `` Only a relative handful of such reports was received '' , the jury said , `` considering the widespread interest in the election , the number of voters and the size of this city '' .
# 2. Nevertheless , `` we feel that in the future Fulton County should receive some portion of these available funds '' , the jurors said .
# 3. -- After a long , hot controversy , Miller County has a new school superintendent , elected , as a policeman put it , in the `` coolest election I ever saw in this county '' .
# 4. `` This was the coolest , calmest election I ever saw '' , Colquitt Policeman Tom Williams said .
# 5. `` Everything went real smooth '' , the sheriff said .
 

1. `` Only a relative handful of such reports was received '' , the jury said , `` considering the widespread interest in the election , the number of voters and the size of this city '' .
2. Nevertheless , `` we feel that in the future Fulton County should receive some portion of these available funds '' , the jurors said .
3. -- After a long , hot controversy , Miller County has a new school superintendent , elected , as a policeman put it , in the `` coolest election I ever saw in this county '' .
4. `` This was the coolest , calmest election I ever saw '' , Colquitt Policeman Tom Williams said .
5. `` Everything went real smooth '' , the sheriff said .


Conclusions:

1.TextRank uses the intuition behind the PageRank algorithm to rank sentences

2.TextRank is an unsupervised method for computing the extractive summary of a text. No training is necessary.

3.Given that it’s language independent, it an be used on any language, not only English.

4.In the above exercise, we have used the cosine similarity as the similarity measure. We have also converted sentences to vectors to compute this similarity.